## Playing around with Spark

In [24]:
from pyspark.sql import SparkSession

In [8]:
spark = SparkSession.builder.appName('mylog').getOrCreate()

In [47]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [25]:
#from pyspark.ml.classification import LogisticRegression

In [48]:
#my_data = spark.read.format('csv').load('assessment_data.csv')
my_data = spark.read.csv("assessment_data.csv", inferSchema=True,header=True)

In [50]:
my_data.printSchema()

root
 |-- target_var: integer (nullable = true)
 |-- feature_1: double (nullable = true)
 |-- feature_2: integer (nullable = true)
 |-- feature_3: integer (nullable = true)
 |-- feature_4: string (nullable = true)
 |-- feature_5: string (nullable = true)
 |-- feature_6: double (nullable = true)
 |-- feature_7: integer (nullable = true)
 |-- feature_8: string (nullable = true)
 |-- feature_9: double (nullable = true)



In [52]:
my_data.count()

1309

In [49]:
my_data.describe().show()

+-------+-------------------+-----------------+------------------+------------------+---------+---------+------------------+------------------+---------+------------------+
|summary|         target_var|        feature_1|         feature_2|         feature_3|feature_4|feature_5|         feature_6|         feature_7|feature_8|         feature_9|
+-------+-------------------+-----------------+------------------+------------------+---------+---------+------------------+------------------+---------+------------------+
|  count|               1309|             1308|              1309|              1309|      295|     1309|              1046|              1309|     1307|              1309|
|   mean| 0.3819709702062643|33.29547928134572|0.3850267379679144|0.4988540870893812|     null|     null|  29.8811345124283| 2.294881588999236|     null|101.79412006638843|
| stddev|0.48605517086648325|51.75866823917421|0.8655602753495143|1.0416583905961012|     null|     null|14.413499699923596|0.837836018

In [53]:
my_data.columns

['target_var',
 'feature_1',
 'feature_2',
 'feature_3',
 'feature_4',
 'feature_5',
 'feature_6',
 'feature_7',
 'feature_8',
 'feature_9']

In [54]:
from pyspark.ml.feature import VectorAssembler

In [55]:
assembler = VectorAssembler(inputCols=['feature_1',
                                       'feature_2',
                                       'feature_3',
                                       'feature_6',
                                       'feature_7',
                                       'feature_9'], outputCol='features')

Exception ignored in: <object repr() failed>
Traceback (most recent call last):
  File "/home/ubuntu/anaconda3/lib/python3.6/site-packages/pyspark/ml/wrapper.py", line 105, in __del__
    SparkContext._active_spark_context._gateway.detach(self._java_obj)
AttributeError: 'LogisticRegression' object has no attribute '_java_obj'


In [56]:
output = assembler.transform(my_data)

In [58]:
output.printSchema()

root
 |-- target_var: integer (nullable = true)
 |-- feature_1: double (nullable = true)
 |-- feature_2: integer (nullable = true)
 |-- feature_3: integer (nullable = true)
 |-- feature_4: string (nullable = true)
 |-- feature_5: string (nullable = true)
 |-- feature_6: double (nullable = true)
 |-- feature_7: integer (nullable = true)
 |-- feature_8: string (nullable = true)
 |-- feature_9: double (nullable = true)
 |-- features: vector (nullable = true)



In [59]:
output.head(1)

[Row(target_var=1, feature_1=211.3375, feature_2=0, feature_3=0, feature_4='bigollo-16', feature_5='blue', feature_6=29.0, feature_7=1, feature_8='sigma', feature_9=112.87794148245936, features=DenseVector([211.3375, 0.0, 0.0, 29.0, 1.0, 112.8779]))]

In [38]:
final_data = output.select('features','target_var')

In [60]:
final_data.printSchema()

root
 |-- features: vector (nullable = true)
 |-- target_var: integer (nullable = true)



In [61]:
train, test = final_data.randomSplit([0.7,0.3])

In [62]:
model = LogisticRegression(labelCol='target_var')

In [63]:
model = model.fit(train)

Py4JJavaError: An error occurred while calling o630.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 18.0 failed 1 times, most recent failure: Lost task 0.0 in stage 18.0 (TID 21, localhost, executor driver): org.apache.spark.SparkException: Failed to execute user defined function($anonfun$3: (struct<feature_1:double,feature_2_double_VectorAssembler_4b0e89ecb3d7df4658ae:double,feature_3_double_VectorAssembler_4b0e89ecb3d7df4658ae:double,feature_6:double,feature_7_double_VectorAssembler_4b0e89ecb3d7df4658ae:double,feature_9:double>) => vector)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.sort_addToSorter$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$GeneratedIterator.processNext(Unknown Source)
	at org.apache.spark.sql.execution.BufferedRowIterator.hasNext(BufferedRowIterator.java:43)
	at org.apache.spark.sql.execution.WholeStageCodegenExec$$anonfun$8$$anon$1.hasNext(WholeStageCode

I am still investigationg how to fit my data :(, I should have come up with a simple basic model instead.